# Azure OpenAI client library for .NET

The Azure OpenAI client library for .NET is a companion to the official [OpenAI client library for .NET](https://github.com/openai/openai-dotnet). The Azure OpenAI library configures a client for use with Azure OpenAI and provides additional strongly typed extension support for request and response models specific to Azure OpenAI scenarios.

In [ ]:
#r "nuget:Azure.AI.OpenAI,2.0.0-beta.5"

## Namespace organization
The library is organized into several namespaces corresponding to OpenAI feature areas. Each namespace contains a corresponding client class.

| Namespace          | Client class        | Notes          |
|--------------------|---------------------|----------------|
| OpenAI.Assistants  | AssistantClient     | [Experimental] |
| OpenAI.Audio       | AudioClient         |                |
| OpenAI.Batch       | BatchClient         | [Experimental]               |
| OpenAI.Chat        | ChatClient          |                |
| OpenAI.Embeddings  | EmbeddingClient     |                |
| OpenAI.FineTuning  | FineTuningClient    |                |
| OpenAI.Files       | FileClient          |                |
| OpenAI.Images      | ImageClient         |                |
| OpenAI.Models      | ModelClient         |                |
| OpenAI.Moderations | ModerationClient    |                |
| OpenAI.VectorStores| VectorStoreClient   | [Experimental] |

Some funcions are marked as experimental, which means that they are not yet fully supported and may change in future releases.

In [40]:
#pragma warning disable OPENAI001

Here is an example of how to create Azure OpenAI client

In [41]:
using Azure.AI.OpenAI;

AzureOpenAIClient azureClient = new(
    new Uri("https://ai-proxy.lab.epam.com"),
    Environment.GetEnvironmentVariable("OPENAI_API_KEY"));

And Now lets test the client with a simple example

In [ ]:
using OpenAI.Chat;

ChatClient chatClient = azureClient.GetChatClient("gpt-4o-2024-08-06");
ChatCompletion completion = await chatClient.CompleteChatAsync("Say 'this is a test.'");

completion

So, now we are ready to test a bit complex example using chat client

In [ ]:
ChatCompletion completion = chatClient.CompleteChat(
    [
        // System messages represent instructions or other guidance about how the assistant should behave
        new SystemChatMessage("You are a helpful assistant that talks like a computer geek."),
        // User messages represent user input, whether historical or the most recen tinput
        new UserChatMessage("Hi, can you help me?"),
        // Assistant messages in a request represent conversation history for responses
        new AssistantChatMessage("You’ve got my full CPU power at your service. What’s the mission today?"),
        new UserChatMessage("What breaks your heart?"),
    ]);

completion.Content[0].Text

Now lets test client with streaming API  

When you request a chat completion, the default behavior is for the server to generate it in its entirety before sending it back in a single response. Consequently, long chat completions can require waiting for several seconds before hearing back from the server. To mitigate this, the OpenAI REST API supports the ability to stream partial results back as they are being generated, allowing you to start processing the beginning of the completion before it is finished.

In [ ]:
using System.ClientModel;

CollectionResult<StreamingChatCompletionUpdate> updates = chatClient.CompleteChatStreaming("Say 'this is a test.'");

updates

As you see we need to use a bit different approach for streamed responses.

In [ ]:
CollectionResult<StreamingChatCompletionUpdate> updates = chatClient.CompleteChatStreaming("Say 'this is a test.'");

foreach (StreamingChatCompletionUpdate completionUpdate in updates)
{
    foreach (ChatMessageContentPart contentPart in completionUpdate.ContentUpdate)
    {
        Console.WriteLine(contentPart.Text);
    }
}

### Text Embeddings

Let's imagine you have a list of sentences and you want to find the most similar sentence to a given query. You can use the `EmbeddingClient` to get the embeddings of the sentences and then calculate the cosine similarity between the query and each sentence.

In [ ]:
using OpenAI.Embeddings;

EmbeddingClient embeddingsClient = azureClient.GetEmbeddingClient("text-embedding-ada-002");

string description = @"Best hotel in town if you like luxury hotels. They have an amazing infinity pool, a spa,
    and a really helpful concierge. The location is perfect -- right downtown, close to all the tourist
    attractions. We highly recommend this hotel.";

Embedding embedding = embeddingsClient.GenerateEmbedding(description);
ReadOnlyMemory<float> vector = embedding.Vector;

vector

### How to use chat completions with structured outputs

Beginning with the gpt-4o-mini, gpt-4o-mini-2024-07-18, and gpt-4o-2024-08-06 model snapshots, structured outputs are available for both top-level response content and tool calls in the chat completion and assistants APIs.

For information about the feature, see the [Structured Outputs guide](https://platform.openai.com/docs/guides/structured-outputs/introduction).


In [ ]:
using System.Text.Json;

ChatCompletionOptions options = new()
{
    ResponseFormat = ChatResponseFormat.CreateJsonSchemaFormat(
        name: "math_reasoning",
        jsonSchema: BinaryData.FromString("""
            {
                "type": "object",
                "properties": {
                "steps": {
                    "type": "array",
                    "items": {
                    "type": "object",
                    "properties": {
                        "explanation": { "type": "string" },
                        "output": { "type": "string" }
                    },
                    "required": ["explanation", "output"],
                    "additionalProperties": false
                    }
                },
                "final_answer": { "type": "string" }
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": false
            }
            """),
    strictSchemaEnabled: true)
};

ChatCompletion chatCompletion = await chatClient.CompleteChatAsync( ["How can I solve 8x + 7 = -23?"],  options);

var structuredJson = JsonDocument.Parse(chatCompletion.ToString());
        
Console.WriteLine($"Final answer: {structuredJson.RootElement.GetProperty("final_answer").GetString()}");
Console.WriteLine("Reasoning steps:");

foreach (JsonElement stepElement in structuredJson.RootElement.GetProperty("steps").EnumerateArray())
{
    Console.WriteLine($"  - Explanation: {stepElement.GetProperty("explanation").GetString()}");
    Console.WriteLine($"    Output: {stepElement.GetProperty("output")}");
}

### TODO: Fix the code below

In [ ]:
using OpenAI.Images;

ImageClient imageClient = azureClient.GetImageClient("dall-e-3");

string prompt = @"The concept for a living room that blends Scandinavian simplicity with Japanese minimalism for
     a serene and cozy atmosphere. It's a space that invites relaxation and mindfulness, with natural light
     and fresh air. Using neutral tones, including colors like white, beige, gray, and black, that create a
     sense of harmony. Featuring sleek wood furniture with clean lines and subtle curves to add warmth and
     elegance. Plants and flowers in ceramic pots adding color and life to a space. They can serve as focal
     points, creating a connection with nature. Soft textiles and cushions in organic fabrics adding comfort
     and softness to a space. They can serve as accents, adding contrast and texture.";

ImageGenerationOptions options = new()
{
    Quality = GeneratedImageQuality.Standard,
    Size = GeneratedImageSize.W256xH256,
    Style = GeneratedImageStyle.Natural,
    ResponseFormat = GeneratedImageFormat.Uri
};

GeneratedImage image = imageClient.GenerateImage(prompt, options);

image.ImageUri